In [73]:
import sys

# Add the parent directory to the system path
sys.path.append(r'C:\Users\kaise\OneDrive\Desktop\Codebase\FYP\pkg\utils')

In [74]:
import re
import requests
import geopandas as gpd
import numpy as np
import pandas as pd
from read_df_from_s3 import read_df_from_s3
from motherduckdb_connector import connect_to_motherduckdb, MotherDuckDBConnector
from shapely.geometry import Point
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime

date = "2024-07-31"
df = read_df_from_s3(f"airflow/propnex/{date}.parquet.gzip")
print(f"Dataframe downloaded with shape {df.shape}")

Dataframe downloaded with shape (753, 21)


In [75]:
# from find_closest import find_nearest

# def fetch_mrt_info(db: MotherDuckDBConnector):
#     df2 = db.query_df("SELECT * FROM mrt_info")

#     print(f"mrt_info: \n{df2.head()}\n")

#     df2 = df2[["station_name", "latitude", "longitude"]]
#     return df2

# db = connect_to_motherduckdb()
# mrt_info = fetch_mrt_info(db)
# db.close()

# Data Exploration

In [76]:
df.head()

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,latitude,longitude,...,floor_level,furnishing,facing,built_year,tenure,property_type,nearest_mrt,distance_to_nearest_mrt,url,facilities
0,Robinson Suites,510312,Boat Quay / Raffles Place / Marina,"$4,200",2.0,1.0,614,Robinson Road,1.2813,103.8528,...,High Floor,None,NaN,NaN,None,Condominium,NaN,NaN,https://www.propnex.com/listing-details/510312...,None
1,V on Shenton,403782,Boat Quay / Raffles Place / Marina,"$7,300",2.0,2.0,1216,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/403782...,None
2,One Shenton,534500,Boat Quay / Raffles Place / Marina,"$6,500",2.0,2.0,1119,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/534500...,None
3,The Sail @ Marina Bay,557637,Boat Quay / Raffles Place / Marina,"$6,299",2.0,1.0,657,Marina Boulevard,1.2813,103.8528,...,Middle Floor,Fully Furnished,NaN,NaN,None,Apartment,NaN,NaN,https://www.propnex.com/listing-details/557637...,"['City View', 'Cooker Hob/Hood', 'Air conditio..."
4,One Shenton,376694,Boat Quay / Raffles Place / Marina,"$9,000",3.0,3.0,1593,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/376694...,None


In [77]:
df.isna().sum()

property_name                0
listing_id                   0
district                     0
price                        0
bedroom                     40
bathroom                    43
dimensions                   0
address                     12
latitude                   463
longitude                  463
price/sqft                   0
floor_level                494
furnishing                 495
facing                     753
built_year                 753
tenure                     110
property_type                0
nearest_mrt                753
distance_to_nearest_mrt    753
url                          0
facilities                 478
dtype: int64

There's no `facing`, `built_year`, `nearest_mrt` and `distance_to_nearest_mrt`.

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   property_name            753 non-null    object 
 1   listing_id               753 non-null    object 
 2   district                 753 non-null    object 
 3   price                    753 non-null    object 
 4   bedroom                  713 non-null    float64
 5   bathroom                 710 non-null    float64
 6   dimensions               753 non-null    int64  
 7   address                  741 non-null    object 
 8   latitude                 290 non-null    float64
 9   longitude                290 non-null    float64
 10  price/sqft               753 non-null    object 
 11  floor_level              259 non-null    object 
 12  furnishing               258 non-null    object 
 13  facing                   0 non-null      float64
 14  built_year               0

In [79]:
len(df[df.duplicated(subset='listing_id', keep=False)])

0

In [80]:
df[df['latitude'].isna() | df['longitude'].isna()]

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,latitude,longitude,...,floor_level,furnishing,facing,built_year,tenure,property_type,nearest_mrt,distance_to_nearest_mrt,url,facilities
1,V on Shenton,403782,Boat Quay / Raffles Place / Marina,"$7,300",2.0,2.0,1216,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/403782...,None
2,One Shenton,534500,Boat Quay / Raffles Place / Marina,"$6,500",2.0,2.0,1119,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/534500...,None
4,One Shenton,376694,Boat Quay / Raffles Place / Marina,"$9,000",3.0,3.0,1593,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/376694...,None
5,One Shenton,556819,Boat Quay / Raffles Place / Marina,"$4,500",1.0,1.0,538,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/556819...,None
6,V on Shenton,550956,Boat Quay / Raffles Place / Marina,"$7,900",2.0,2.0,1206,Shenton Way,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Apartment,NaN,NaN,https://www.propnex.com/listing-details/550956...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,High Park Residences,447248,Seletar / Yio Chu Kang,"$1,150",1.0,2.0,872,Fernvale Road,NaN,NaN,...,Low Floor,Fully Furnished,NaN,NaN,None,Condominium,NaN,NaN,https://www.propnex.com/listing-details/447248...,"['24 hours security', 'Bath Room', 'Living Roo..."
740,The Greenwich,552054,Seletar / Yio Chu Kang,"$3,000",1.0,1.0,603,Seletar Road,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/552054...,None
741,The Greenwich,347082,Seletar / Yio Chu Kang,"$3,800",2.0,2.0,883,Seletar Road,NaN,NaN,...,None,None,NaN,NaN,Leasehold,Condominium,NaN,NaN,https://www.propnex.com/listing-details/347082...,None
750,RiverTrees Residences,470688,Seletar / Yio Chu Kang,"$3,250",2.0,2.0,840,Fernvale Close,NaN,NaN,...,None,None,NaN,NaN,None,Apartment,NaN,NaN,https://www.propnex.com/listing-details/470688...,None


In [81]:
df['floor_level'].unique()

array(['High Floor', None, 'Middle Floor', 'Low Floor', 'Penthouse',
       'Ground Floor'], dtype=object)

In [82]:
df['facing'].unique()

array([nan])

In [83]:
df['furnishing'].unique()

array([None, 'Fully Furnished', 'Partially Furnished', 'Unfurnished'],
      dtype=object)

In [84]:
df['tenure'].unique()

array([None, 'Leasehold', '99-year Leasehold', 'Freehold',
       '999-year Leasehold', '103-year Leasehold'], dtype=object)

In [85]:
df['property_type'].unique()

array(['Condominium', 'Apartment'], dtype=object)

In [86]:
df['property_name']

0            Robinson Suites
1               V on Shenton
2                One Shenton
3      The Sail @ Marina Bay
4                One Shenton
               ...          
748    Riverbank at Fernvale
749           H2O Residences
750    RiverTrees Residences
751             Grande Vista
752              The Topiary
Name: property_name, Length: 753, dtype: object

In [87]:
df["building_name"] = df["property_name"]

# Data Preprocessing

Drop rows with no address and coordinate (can't be saved)

In [88]:
no_addr_and_coord = df[(df['latitude'].isna() | df['longitude'].isna()) & df['address'].isna()]
len(no_addr_and_coord)

7

In [89]:
# drop them
df.drop(no_addr_and_coord.index, inplace=True)

In [90]:
def fetch_coordinates(location_name):
    url = 'https://www.onemap.gov.sg/api/common/elastic/search'
    params = {
        'searchVal': location_name,
        'returnGeom': 'Y',
        'getAddrDetails': 'Y',
        'pageNum': 1
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['found'] > 0:
            return location_name, (data['results'][0]['LATITUDE'], data['results'][0]['LONGITUDE'])
    print(f"Failed to fetch coordinates for {location_name}")
    return location_name, (np.nan, np.nan)

def update_lat_long_v2(df):
    # Locate rows with null lat and long
    df_null_coords = df[(df['latitude'].isnull()) & (df['longitude'].isnull())]
    addresses = df_null_coords['address'].unique()

    with ThreadPoolExecutor() as executor:
        print(f"Using {executor._max_workers} threads to fetch coordinates\n")
        future_to_coords = {executor.submit(
            fetch_coordinates, name): name for name in addresses}
        for future in as_completed(future_to_coords):
            result = future.result()
            if result is None:
                continue

            address, coords = result
            df.loc[df['address'] ==
                   address, 'latitude'] = coords[0]
            df.loc[df['address'] ==
                   address, 'longitude'] = coords[1]

    return df

In [91]:
df = update_lat_long_v2(df)

Using 12 threads to fetch coordinates



In [92]:
no_coord = df[df['latitude'].isna() | df['longitude'].isna()]
len(no_coord)

0

In [93]:
from location_constants import DISTRICTS
REVERSE_DISTRICTS = {v: k for k, v in DISTRICTS.items()}

df["district_id"] = df["district"].map(REVERSE_DISTRICTS)

In [94]:
from find_closest import find_nearest

def fetch_mrt_info(db: MotherDuckDBConnector):
    df2 = db.query_df("SELECT * FROM mrt_info")

    print(f"mrt_info: \n{df2.head()}\n")

    df2 = df2[["station_name", "latitude", "longitude"]]
    return df2

def update_mrt(df):
    df_null_mrt = df[df["nearest_mrt"].isnull()]
    
    db = connect_to_motherduckdb()
    mrt_info = fetch_mrt_info(db)
    db.close()
    df_null_mrt = find_nearest(
        df_null_mrt, mrt_info, "nearest_mrt", "distance_to_nearest_mrt")
    df.update(df_null_mrt)

    return df

df = update_mrt(df)

mrt_info: 
  station_name   longitude  latitude station_code   color         line
0    Esplanade  103.855612  1.293260          CC3  Orange       Circle
1   Paya Lebar  103.892272  1.317369          EW8   Green    East West
2   Paya Lebar  103.892272  1.317369          CC9  Orange       Circle
3  Dhoby Ghaut  103.845833  1.299044         NS24     Red  North South
4  Dhoby Ghaut  103.845833  1.299044          NE6  Purple   North East



In [95]:
df["nearest_hawker"] = None
df["distance_to_nearest_hawker"] = float("inf")

def fetch_hawker_info(db: MotherDuckDBConnector):
    df2 = db.query_df("SELECT * FROM hawker_centre_info")

    print(f"hawker_centre_info: \n{df2.head()}\n")

    df2 = df2[["name", "latitude", "longitude"]]
    return df2

def update_hawker(df):
    df_null_hawker = df[df["nearest_hawker"].isnull()]
    
    db = connect_to_motherduckdb()
    hawker_info = fetch_hawker_info(db)
    db.close()
    df_null_hawker = find_nearest(
        df_null_hawker, hawker_info, "nearest_hawker", "distance_to_nearest_hawker")
    df.update(df_null_hawker)

    return df

df = update_hawker(df)

hawker_centre_info: 
   hawker_id                             name  \
0          1      Market Street Hawker Centre   
1          2     Marsiling Mall Hawker Centre   
2          3     Margaret Drive Hawker Centre   
3          4  Fernvale Hawker Centre & Market   
4          5        One Punggol Hawker Centre   

                                     building_name           street_name  \
0                                     CapitaSpring         Market Street   
1                                   Marsiling Mall   Woodlands Street 12   
2                                                         Margaret Drive   
3  Fernvale Community Club, Hawker Centre & Market  Sengkang West Avenue   
4                                      One Punggol         Punggol Drive   

  postal_code   longitude  latitude  
0       48940  103.850165  1.284425  
1      738623  103.779785  1.433539  
2      142038  103.804715  1.297486  
3      797650  103.877060  1.391592  
4      828629  103.904806  1.408190  

In [96]:
df["nearest_supermarket"] = None
df["distance_to_nearest_supermarket"] = float("inf")

def fetch_supermarket_info(db: MotherDuckDBConnector):
    df2 = db.query_df("SELECT * FROM supermarket_info")

    print(f"supermarket_info: \n{df2.head()}\n")

    df2 = df2[["name", "latitude", "longitude"]]
    return df2

def update_supermarket(df):
    df_null_supermarket = df[df["nearest_supermarket"].isnull()]

    db = connect_to_motherduckdb()
    supermarket_info = fetch_supermarket_info(db)
    db.close()
    df_null_supermarket = find_nearest(
        df_null_supermarket, supermarket_info, "nearest_supermarket", "distance_to_nearest_supermarket")
    df.update(df_null_supermarket)

    return df

df = update_supermarket(df)

supermarket_info: 
   supermarket_id                                         name  \
0               1  LI LI CHENG SUPERMARKET (PUNGGOL) PTE. LTD.   
1               2              SHENG SIONG SUPERMARKET PTE LTD   
2               3        COLD STORAGE SINGAPORE (1983) PTE LTD   
3               4        COLD STORAGE SINGAPORE (1983) PTE LTD   
4               5                      YES SUPERMARKET PTE LTD   

            street_name postal_code   longitude  latitude  
0         PUNGGOL PLACE      823273  103.901262  1.402303  
1  UPPER BOON KENG ROAD      380011  103.870914  1.314239  
2      HOUGANG AVENUE 8      530683  103.886366  1.373321  
3  BEDOK RESERVOIR ROAD      470631  103.914942  1.332959  
4    TAMPINES STREET 21      522201  103.953010  1.353453  



In [97]:
df["nearest_sch"] = None
df["distance_to_nearest_sch"] = float("inf")

def fetch_primary_school_info(db: MotherDuckDBConnector):
    df2 = db.query_df("SELECT * FROM primary_school_info")

    print(f"primary_school_info: \n{df2.head()}\n")

    df2 = df2[["name", "latitude", "longitude"]]
    return df2

def update_primary_school(df):
    df_null_sch = df[df["nearest_sch"].isnull()]
    
    db = connect_to_motherduckdb()
    sch_info = fetch_primary_school_info(db)
    db.close()
    df_null_sch = find_nearest(
        df_null_sch, sch_info, "nearest_sch", "distance_to_nearest_sch")
    df.update(df_null_sch)

    return df


df = update_primary_school(df)

primary_school_info: 
   school_id                          name         area   longitude  latitude
0          1      Admiralty Primary School    Woodlands  103.800214  1.442550
1          2  Ahmad Ibrahim Primary School       Yishun  103.832942  1.433153
2          3                Ai Tong School       Bishan  103.833020  1.360583
3          4      Alexandra Primary School  Bukit Merah  103.824425  1.291334
4          5   Anchor Green Primary School     Sengkang  103.887165  1.390370



In [98]:
df["nearest_mall"] = None
df["distance_to_nearest_mall"] = float("inf")

def fetch_mall_info(db: MotherDuckDBConnector):
    df2 = db.query_df("SELECT * FROM mall_info")

    print(f"mall_info: \n{df2.head()}\n")

    df2 = df2[["name", "latitude", "longitude"]]
    return df2

def update_mall(df):
    df_null_mall = df[df["nearest_mall"].isnull()]
    
    db = connect_to_motherduckdb()
    mall_info = fetch_mall_info(db)
    db.close()
    df_null_mall = find_nearest(
        df_null_mall, mall_info, "nearest_mall", "distance_to_nearest_mall")
    df.update(df_null_mall)

    return df

df = update_mall(df)

mall_info: 
   mall_id                            name   longitude  latitude
0        1                          100 AM  103.843471  1.274588
1        2                    313@Somerset  103.838361  1.301014
2        3                          Aperia  103.863930  1.310867
3        4  Balestier Hill Shopping Centre  103.842572  1.325596
4        5                      Bugis Cube  103.855635  1.298141



In [99]:
def simplify_lease_type(lease_type):
    if pd.isnull(lease_type):
        return None
    elif 'leasehold' in lease_type.lower():
        return 'leasehold'
    else:
        return 'freehold'
    
df['property_type'] = df['property_type'].astype('category')
df['furnishing'] = df['furnishing'].fillna('Unfurnished').astype('category')
df['tenure'] = df['tenure'].apply(simplify_lease_type).astype('category')
df['floor_level'] = df['floor_level'].astype('category')
df['district'] = df['district'].astype('category')

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746 entries, 0 to 752
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   property_name                    746 non-null    object  
 1   listing_id                       746 non-null    object  
 2   district                         746 non-null    category
 3   price                            746 non-null    object  
 4   bedroom                          708 non-null    float64 
 5   bathroom                         704 non-null    float64 
 6   dimensions                       746 non-null    int64   
 7   address                          741 non-null    object  
 8   latitude                         746 non-null    object  
 9   longitude                        746 non-null    object  
 10  price/sqft                       746 non-null    object  
 11  floor_level                      258 non-null    category
 12  furnishi

In [101]:
df.head()

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,latitude,longitude,...,building_name,district_id,nearest_hawker,distance_to_nearest_hawker,nearest_supermarket,distance_to_nearest_supermarket,nearest_sch,distance_to_nearest_sch,nearest_mall,distance_to_nearest_mall
0,Robinson Suites,510312,Boat Quay / Raffles Place / Marina,"$4,200",2.0,1.0,614,Robinson Road,1.2813,103.8528,...,Robinson Suites,D01,Market Street Hawker Centre,453.211,AL MARCHE PTE LTD,60.776,Tao Nan School,1500.975,Marina Bay Link Mall,198.705
1,V on Shenton,403782,Boat Quay / Raffles Place / Marina,"$7,300",2.0,2.0,1216,Shenton Way,1.27612402388542,103.848159371383,...,V on Shenton,D01,Amoy Street Food Centre (Telok Ayer Food Centre),383.264,LITTLE FARMS PTE. LTD.,340.902,Cantonment Primary School,915.027,Square 2,376.025
2,One Shenton,534500,Boat Quay / Raffles Place / Marina,"$6,500",2.0,2.0,1119,Shenton Way,1.27612402388542,103.848159371383,...,One Shenton,D01,Amoy Street Food Centre (Telok Ayer Food Centre),383.264,LITTLE FARMS PTE. LTD.,340.902,Cantonment Primary School,915.027,Square 2,376.025
3,The Sail @ Marina Bay,557637,Boat Quay / Raffles Place / Marina,"$6,299",2.0,1.0,657,Marina Boulevard,1.28537030528984,103.860871154701,...,The Sail @ Marina Bay,D01,Market Street Hawker Centre,1196.082,COLD STORAGE SINGAPORE (1983) PTE LTD,330.757,Tao Nan School,1649.176,Marina Bay Sands,212.673
4,One Shenton,376694,Boat Quay / Raffles Place / Marina,"$9,000",3.0,3.0,1593,Shenton Way,1.27612402388542,103.848159371383,...,One Shenton,D01,Amoy Street Food Centre (Telok Ayer Food Centre),383.264,LITTLE FARMS PTE. LTD.,340.902,Cantonment Primary School,915.027,Square 2,376.025


In [102]:
def extract_num_price(x):
    if not x:
        return ""
    res = re.findall(r'\d[\d,]*', x)
    return res[0] if res else ""

In [103]:
df['price'] = df['price'].apply(
    extract_num_price).str.replace(',', '').astype(int)

In [104]:
df[df['bedroom'] == 1]

,property_name,listing_id,district,price,bedroom,bathroom,dimensions,address,latitude,longitude,...,building_name,district_id,nearest_hawker,distance_to_nearest_hawker,nearest_supermarket,distance_to_nearest_supermarket,nearest_sch,distance_to_nearest_sch,nearest_mall,distance_to_nearest_mall
5,One Shenton,556819,Boat Quay / Raffles Place / Marina,4500,1.0,1.0,538,Shenton Way,1.27612402388542,103.848159371383,...,One Shenton,D01,Amoy Street Food Centre (Telok Ayer Food Centre),383.264,LITTLE FARMS PTE. LTD.,340.902,Cantonment Primary School,915.027,Square 2,376.025
8,Marina One Residences,518532,Boat Quay / Raffles Place / Marina,4800,1.0,1.0,721,Marina Way,1.27832254934626,103.854609590898,...,Marina One Residences,D01,Market Street Hawker Centre,836.660,COLD STORAGE SINGAPORE (1983) PTE LTD,222.743,Cantonment Primary School,1660.223,Marina Bay Link Mall,211.815
10,Marina One Residences,556560,Boat Quay / Raffles Place / Marina,5000,1.0,1.0,743,Marina Way,1.27832254934626,103.854609590898,...,Marina One Residences,D01,Market Street Hawker Centre,836.660,COLD STORAGE SINGAPORE (1983) PTE LTD,222.743,Cantonment Primary School,1660.223,Marina Bay Link Mall,211.815
11,Marina One Residences,114500,Boat Quay / Raffles Place / Marina,4900,1.0,1.0,721,Marina Way,1.27832254934626,103.854609590898,...,Marina One Residences,D01,Market Street Hawker Centre,836.660,COLD STORAGE SINGAPORE (1983) PTE LTD,222.743,Cantonment Primary School,1660.223,Marina Bay Link Mall,211.815
12,The Sail @ Marina Bay,555790,Boat Quay / Raffles Place / Marina,1599,1.0,2.0,230,Marina Boulevard,1.28537030528984,103.860871154701,...,The Sail @ Marina Bay,D01,Market Street Hawker Centre,1196.082,COLD STORAGE SINGAPORE (1983) PTE LTD,330.757,Tao Nan School,1649.176,Marina Bay Sands,212.673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,Parc Botannia,322437,Seletar / Yio Chu Kang,3000,1.0,1.0,506,Fernvale Street,1.39844234483729,103.879092907812,...,Parc Botannia,D28,Fernvale Hawker Centre & Market,790.540,SHENG SIONG SUPERMARKET PTE LTD,298.620,Sengkang Green Primary School,754.247,The Seletar Mall,838.693
739,High Park Residences,447248,Seletar / Yio Chu Kang,1150,1.0,2.0,872,Fernvale Road,1.39350099527635,103.875847562244,...,High Park Residences,D28,Fernvale Hawker Centre & Market,250.526,NTUC FAIRPRICE CO-OPERATIVE LTD,234.379,Sengkang Green Primary School,100.768,The Seletar Mall,226.705
740,The Greenwich,552054,Seletar / Yio Chu Kang,3000,1.0,1.0,603,Seletar Road,1.41296690976325,103.872710743136,...,The Greenwich,D28,Fernvale Hawker Centre & Market,2412.579,LABIBA ENTERPRISE PTE. LTD.,1812.713,Sengkang Green Primary School,2242.296,The Seletar Mall,2407.213
748,Riverbank at Fernvale,508161,Seletar / Yio Chu Kang,2500,1.0,1.0,495,Sengkang West Way/Fernvale,1.394393,103.879822,...,Riverbank at Fernvale,D28,Fernvale Hawker Centre & Market,436.331,SHENG SIONG SUPERMARKET PTE LTD,310.290,Sengkang Green Primary School,540.155,The Seletar Mall,523.091


In [105]:
if "is_whole_unit" not in df.columns:
    from bs4 import BeautifulSoup
    def get_listing_type(url):
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        prop_list_box = soup.find_all("div", class_="property-list-box")
        
        for box in prop_list_box:
            labels = box.find_all("b")
            values = box.find_all("span")
            
            for col, val in zip(labels, values):
                label = col.get_text(strip=True)
                if label != "Listing Type":
                    continue
                listing_type = val.get_text(strip=True)
                is_whole_unit = True if listing_type != "ROOM" else False
                return is_whole_unit
        return True

    df['is_whole_unit'] = df['url'].apply(get_listing_type)

In [106]:
df['has_pool'] = df['facilities'].apply(lambda x: 'pool' in x.lower() if x else False)
df['has_gym'] = df['facilities'].apply(lambda x: 'gym' in x.lower() if x else False)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746 entries, 0 to 752
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   property_name                    746 non-null    object  
 1   listing_id                       746 non-null    object  
 2   district                         746 non-null    category
 3   price                            746 non-null    int32   
 4   bedroom                          708 non-null    float64 
 5   bathroom                         704 non-null    float64 
 6   dimensions                       746 non-null    int64   
 7   address                          741 non-null    object  
 8   latitude                         746 non-null    object  
 9   longitude                        746 non-null    object  
 10  price/sqft                       746 non-null    object  
 11  floor_level                      258 non-null    category
 12  furnishi

In [108]:
df['is_whole_unit'].fillna(False).value_counts()

True     688
False     58
Name: is_whole_unit, dtype: int64

In [109]:
df["fingerprint"] = df["listing_id"] + "-" + df["price"].astype(str)
df["fingerprint"]

0      510312-4200
1      403782-7300
2      534500-6500
3      557637-6299
4      376694-9000
          ...     
748    508161-2500
749    539645-2880
750    470688-3250
751    400941-4800
752    529293-3400
Name: fingerprint, Length: 746, dtype: object

In [110]:
df['source'] = 'propnex'
df['scraped_on'] = datetime.strptime(date, '%Y-%m-%d')
df['last_updated'] = df['scraped_on']

df.rename(columns={
    'price/sqft': 'price_per_sqft',
    'distance_to_nearest_mrt': 'distance_to_mrt_in_m',
    'distance_to_nearest_hawker': 'distance_to_hawker_in_m',
    'distance_to_nearest_sch': 'distance_to_sch_in_m',
    'distance_to_nearest_supermarket': 'distance_to_supermarket_in_m',
    'distance_to_nearest_mall': 'distance_to_mall_in_m',
}, inplace=True)

In [111]:
df.describe()

,price,bedroom,bathroom,dimensions,facing,built_year,distance_to_mrt_in_m,distance_to_hawker_in_m,distance_to_supermarket_in_m,distance_to_sch_in_m,distance_to_mall_in_m
count,7.460000e+02,708.000000,704.000000,746.000000,0.0,0.0,746.000000,746.000000,746.000000,746.000000,746.000000
mean,8.444983e+03,2.091808,1.892045,880.876676,NaN,NaN,603.015050,818.486261,388.601693,715.917712,711.042831
std,7.355608e+04,0.987211,0.934403,646.169895,NaN,NaN,466.368901,591.026884,294.741314,435.520039,551.925756
min,8.500000e+02,1.000000,1.000000,70.000000,NaN,NaN,30.846000,0.052000,0.018000,90.191000,7.328000
25%,3.300000e+03,1.000000,1.000000,517.250000,NaN,NaN,260.186000,406.932500,204.328000,438.524000,273.423000
50%,4.200000e+03,2.000000,2.000000,748.000000,NaN,NaN,479.716000,692.670000,329.505000,627.025000,553.797000
75%,5.300000e+03,3.000000,2.000000,1109.000000,NaN,NaN,772.212000,1036.241750,493.318250,915.027000,1063.994000
max,1.779000e+06,6.000000,6.000000,6393.000000,NaN,NaN,3258.649000,3885.272000,2137.556000,3731.153000,3098.651000


In [112]:
df["price"]

0      4200
1      7300
2      6500
3      6299
4      9000
       ... 
748    2500
749    2880
750    3250
751    4800
752    3400
Name: price, Length: 746, dtype: int32

In [113]:
from location_constants import PROPERTY_LISTING_COLS, RENTAL_PRICE_HISTORY_COLS
df = df[PROPERTY_LISTING_COLS]

ImportError: cannot import name 'PROPERTY_LISTING_COLS' from 'location_constants' (C:\Users\kaise\OneDrive\Desktop\Codebase\FYP\pkg\utils\location_constants.py)